In [ ]:
!pip install sentinelsat
!pip install ipywidgets
!pip install --upgrade attrs
!pip install h5netcdf
!pip install netCDF4
!pip install scipy
!pip uninstall xarray -y
!pip install xarray
!pip install matplotlib
!conda install -c conda-forge harp -y

from sentinelsat import SentinelAPI, read_geojson, geojson_to_wkt
import ipywidgets
import numpy as np
from glob import iglob
import harp

import xarray as xr
import numpy as np
from os.path import join
from matplotlib import pyplot as plt
import os
from datetime import date, timedelta

In [ ]:
api = SentinelAPI('s5pguest', 's5pguest', 'https://s5phub.copernicus.eu/dhus/')
#https://s5phub.copernicus.eu/dhus/#/home
#https://apihub.copernicus.eu/apihub

footprint = geojson_to_wkt(read_geojson('./map-wide-delhi.geojson'))

start_date = date(2018, 4, 30)

end_date = date(2022, 7, 19)
count = 0
delta = timedelta(days=1)

while start_date <= end_date:
    tmp = start_date
    start_date += delta
    products = api.query(footprint,
                     date=(tmp, start_date),
                     platformname='Sentinel-5 Precursor',
                    producttype='L2__NO2___',
                    limit=1)
    if len(products) == 0:
        count += 1
        print(count, "missing!!!")
        continue
        
    api.download_all(products)
    s5p_file = ''
    for key, value in products.items():
        #print(key, value)
        print(value['title'])
        s5p_file = value['title']
        s5p_file1 = value['title'] + '.nc'
    
    export_path = ""
    filename = s5p_file1
    harp_L2_L3 = harp.import_product(filename, operations=" \
                                                tropospheric_NO2_column_number_density_validity>50\
                                                derive(tropospheric_NO2_column_number_density [Pmolec/cm2]); \
                                                derive(tropospheric_NO2_column_number_density_validity);\
                                                bin_spatial(83, 26.55, 0.05, 82, 75.1, 0.05); \
                                                derive(latitude {latitude}); derive(longitude {longitude})")
    filename = filename.split('/')[-1].replace('L2', '_wide_L3')
    export_folder = "{export_path}/{file}".format(export_path=export_path, file=filename)
    harp.export_product(harp_L2_L3, filename, file_format='netcdf')
    
    #no2_subset_nparray = no2_subset.to_numpy()
    #np.save(s5p_file, no2_subset_nparray)
    os.remove(s5p_file1)
    #no2_subset_tensor = torch.from_numpy(no2_subset_nparray)